## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# https://stackoverflow.com/questions/59228094/module-not-found-error-no-module-named-config
import sys
sys.path.append('../')
import config

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,51.12,66,56,9.22,broken clouds
1,1,Mackay,AU,-21.1500,149.2000,68.04,94,2,4.61,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,46.47,82,39,4.21,scattered clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,44.26,76,75,4.61,light intensity drizzle
4,4,Butaritari,KI,3.0707,172.7902,82.81,70,16,10.22,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Butaritari,KI,3.0707,172.7902,82.81,70,16,10.22,few clouds
10,10,Shieli,KZ,44.1667,66.7500,76.05,21,16,5.50,few clouds
13,13,Albany,US,42.6001,-73.9662,77.65,59,30,3.38,scattered clouds
14,14,Avarua,CK,-21.2078,-159.7750,75.25,73,75,5.75,light rain
17,17,Cabedelo,BR,-6.9811,-34.8339,75.87,85,1,12.12,clear sky
19,19,Kapaa,US,22.0752,-159.3190,84.52,71,40,1.99,scattered clouds
27,27,Atuona,PF,-9.8000,-139.0333,77.50,70,35,20.76,scattered clouds
35,35,Shingu,JP,33.7333,135.9833,82.90,70,3,3.44,clear sky
37,37,Taitung,TW,22.7583,121.1444,77.27,88,75,3.44,broken clouds
48,48,Banting,MY,2.8167,101.5000,77.14,91,100,3.49,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.isnull().sum())
print(preferred_cities_df.count())

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64
City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Butaritari,KI,82.81,few clouds,3.0707,172.7902,
10,Shieli,KZ,76.05,few clouds,44.1667,66.7500,
13,Albany,US,77.65,scattered clouds,42.6001,-73.9662,
14,Avarua,CK,75.25,light rain,-21.2078,-159.7750,
17,Cabedelo,BR,75.87,clear sky,-6.9811,-34.8339,
19,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
27,Atuona,PF,77.50,scattered clouds,-9.8000,-139.0333,
35,Shingu,JP,82.90,clear sky,33.7333,135.9833,
37,Taitung,TW,77.27,broken clouds,22.7583,121.1444,
48,Banting,MY,77.14,overcast clouds,2.8167,101.5000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Butaritari,KI,82.81,few clouds,3.0707,172.7902,Isles Sunset Lodge
10,Shieli,KZ,76.05,few clouds,44.1667,66.7500,Gostinitsa Aybek Shieli
13,Albany,US,77.65,scattered clouds,42.6001,-73.9662,
14,Avarua,CK,75.25,light rain,-21.2078,-159.7750,Paradise Inn
17,Cabedelo,BR,75.87,clear sky,-6.9811,-34.8339,Hotel Pigalle
19,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Atuona,PF,77.50,scattered clouds,-9.8000,-139.0333,Villa Enata
35,Shingu,JP,82.90,clear sky,33.7333,135.9833,Shingu UI Hotel
37,Taitung,TW,77.27,broken clouds,22.7583,121.1444,Taitung Bali Suites Hotel
48,Banting,MY,77.14,overcast clouds,2.8167,101.5000,Hotel Intan Jugra


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace = True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Butaritari,KI,82.81,few clouds,3.0707,172.7902,Isles Sunset Lodge
10,Shieli,KZ,76.05,few clouds,44.1667,66.7500,Gostinitsa Aybek Shieli
14,Avarua,CK,75.25,light rain,-21.2078,-159.7750,Paradise Inn
17,Cabedelo,BR,75.87,clear sky,-6.9811,-34.8339,Hotel Pigalle
19,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
657,Pa Sang,TH,77.85,overcast clouds,18.5262,98.9394,โรงแรม จามจุรี 99
659,Sioux Lookout,CA,84.27,broken clouds,50.1001,-91.9170,Knobby's Fly-In Camps
661,Tucurui,BR,82.78,scattered clouds,-3.7661,-49.6725,BBB Rooms Centro Tucurui PA
662,Touros,BR,77.90,few clouds,-5.1989,-35.4608,INN NEW HORIZON


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPY_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# Note instructions did not say to add Hotel Name but the image on step 18 showed the hotel name
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))